In [ ]:
from pathlib import Path
import geopandas as gpd
import shutil
from functions_module import load_region_shapefile, load_networks, filter_network_by_region, create_segments_simple, move_overlay_file
from ra2ce.network.network_config_data.network_config_data import NetworkConfigData, NetworkSection, HazardSection
from ra2ce.network.network_config_data.enums.source_enum import SourceEnum
from ra2ce.network.network_config_data.enums.aggregate_wl_enum import AggregateWlEnum
from ra2ce.ra2ce_handler import Ra2ceHandler


In [ ]:
region_list = ["Limburg"]

In [ ]:

region_shapefile = Path(r"P:\bovenregionale-stresstest-hwn\Data\Hazard_maps\Gebiedsindeling obv waterschappen EXPORT 4dec24.gpkg")
HWN_network = Path(r"P:\bovenregionale-stresstest-hwn\Data\Shapes netwerkschakels\Shapes netwerkschakels\HWN_netwerkindeling.shp")
NWB_network = Path(r"P:\bovenregionale-stresstest-hwn\Data\Road_data\Rijkswegen_uit_nwb\rijkswegen.shp")



In [ ]:


for region in region_list:
    print(f"Processing region: {region} hazard map")
    flood_map_folder = Path(rf"P:\bovenregionale-stresstest-hwn\Data\Hazard_maps\{region}\Basisscenario\waterdiepte")
    flood_map_files = list(flood_map_folder.glob("*.tif")) + list(flood_map_folder.glob("*.tiff"))
    if flood_map_files:
        flood_map_path = flood_map_files[0]
        print(f"Using flood map: {flood_map_path}")
    else:
        print("No TIFF files found in the folder.")

    print("Processing networks - clipping to region and joining HWN and NWB networks")
    region_gdf, filtered_region = load_region_shapefile(region_shapefile, region)
    joined_network = load_networks(HWN_network, NWB_network)
    
    print(joined_network.columns)
    code = 'NWSCODE_HWN'  # This is what we use to clip ex: Networkschals inbetween regions should not be clipped
    filtered_network_gdf = filter_network_by_region(joined_network, filtered_region,code)
    
    print("Segmenting networks into 100m segments")
    print("Here we chose the NWB ID as the metric for segmenting the network into 100m segments.")
    segmentation_metric = 'id_NWB' #determine which metric to use for 100m segments
    segmented_gdfs = []
    grouped_networks = filtered_network_gdf.groupby(segmentation_metric)

    for nwscode, group in grouped_networks:
        group_segments = []
        for idx, row in group.iterrows():
            segments_gdf = create_segments_simple(row.geometry, segment_length=100)
            for seg_idx, segment_row in segments_gdf.iterrows():
                new_row = row.copy()
                new_row.geometry = segment_row.geometry
                new_row['segment_id'] = f"{idx}_{seg_idx}"
                new_row['original_id'] = idx
                new_row['segment_number'] = seg_idx
                new_row['segment_length'] = segment_row.geometry.length
                group_segments.append(new_row)
        if group_segments:
            group_gdf = gpd.GeoDataFrame(group_segments, crs=group.crs)
            segmented_gdfs.append(group_gdf)

    if segmented_gdfs:
        segmented_network_gdf = gpd.pd.concat(segmented_gdfs, ignore_index=True)
    else:
        segmented_network_gdf = filtered_network_gdf.copy()

    print("creating inputs for RA2CE run")
    region_dir = Path(rf"P:\bovenregionale-stresstest-hwn\Analysis\{region}")
    input_dir = region_dir.joinpath("Inputs")
    output_dir = region_dir.joinpath("Outputs")
    static_path = input_dir.joinpath("static")
    overlay_path = static_path.joinpath("output_graph")
    network_path = static_path.joinpath("network")
    hazard_path = static_path.joinpath("hazard")
    output_path = output_dir.joinpath("overlay_output")

    for path in [region_dir, input_dir, output_dir, static_path, network_path, hazard_path, output_path]:
        path.mkdir(parents=True, exist_ok=True)

    segmented_network_file = network_path.joinpath(f"{region}_segmented_network.shp")
    columns_to_drop = [col for col in segmented_network_gdf.columns if col.startswith('geometry_') and col != 'geometry']
    segmented_network_clean = segmented_network_gdf.drop(columns=columns_to_drop)
    segmented_network_clean.to_file(segmented_network_file)

    print("converting the flood map to tif from tiff")
    flood_map_filename = flood_map_path.name
    # Convert .tiff extension to .tif if needed
    if flood_map_filename.lower().endswith('.tiff'):
        flood_map_filename = flood_map_filename[:-5] + '.tif'

    destination_path = hazard_path.joinpath(flood_map_filename)
    shutil.copy2(flood_map_path, destination_path)

    root_dir = input_dir
    network_section = NetworkSection(
        source=SourceEnum.SHAPEFILE,
        primary_file=[network_path.joinpath(f"{region}_segmented_network.shp")],
        file_id="REF_ID",
        link_type_column="highway",
        save_gpkg=True
    )

    hazard = HazardSection(
        hazard_map=[Path(file) for file in hazard_path.glob("*.tif")],
        aggregate_wl=AggregateWlEnum.MAX,
        hazard_crs="EPSG:28992"
    )

    network_config_data = NetworkConfigData(
        root_path=root_dir,
        static_path=static_path,
        output_path=output_path,
        network=network_section,
        hazard=hazard
    )

    print("Creating RA2CE handler and running analysis")
    handler = Ra2ceHandler.from_config(network_config_data, None)
    handler.configure()
    handler.run_analysis()

    move_overlay_file(overlay_path, output_path)
